Store Sales - Time Series Forecasting

Step 1: Import Libraries and Load Datasets

In [41]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


# Load datasets - as the date is not the unique index, avoid set up explicit index for further analysis
train = pd.read_csv('data/train.csv', parse_dates=['date'])
test = pd.read_csv('data/test.csv', parse_dates=['date'])
stores = pd.read_csv('data/stores.csv')
oil = pd.read_csv('data/oil.csv', parse_dates=['date'])
holidays = pd.read_csv('data/holidays_events.csv', parse_dates=['date'])
transactions = pd.read_csv('data/transactions.csv', parse_dates=['date'])


# Preview datasets
print("Train Data:")
print(train.head())
print("\nTrain Data Statistcis:")
print(train.describe())
print("\nTrain Data null values:")
print(train.isnull().sum())
print("\nDistinct Date in Train Data:")
print("\nStart date:", train['date'].min())         # Earliest date
print("\nEnd date:", train['date'].max())           # Latest date
print("\nUnique dates:", train['date'].nunique())   # Number of unique dates
print("\nNumber of rows:", train.shape[0])          # Number of rows
print("\nNumber of duplicate rows:", train.duplicated().sum())  # Number of duplicated rows

print("\nTest Data:")
print(test.head())
print("\nTest Data Statistcis:")
print(test.describe())
print("\nTest Data null values:")
print(test.isnull().sum())
print("\nDistinct Date in Test Data:")
print("\nStart date:", test['date'].min())      # Earliest date
print("\nEnd date:", test['date'].max())        # Latest date
print("Unique dates:", test['date'].nunique())  # Number of unique dates
print("\nNumber of rows:", test.shape[0])       # Number of rows
print("\nNumber of duplicate rows:", test.duplicated().sum())   # Number of duplicated rows

print("\nStores Data:")
print(stores.head())
print("\nStores Data Statistcis:")
print(stores.describe())
print("\nStores Data null values:")
print(stores.isnull().sum())
print("Unique stores:", stores['store_nbr'].nunique())  # Number of unique stores
print("Duplicated stores:", stores['store_nbr'].duplicated().sum()) # Number of duplicated stores


print("\nOil Data:")
print(oil.head())
print("\nOil Data Statistcis:")
print(oil.describe())
print("\nOil Data null values:")
print(oil.isnull().sum())
print("\nStart date:", oil['date'].min())       # Earliest date
print("\nEnd date:", oil['date'].max())         # Latest date
print("\nUnique dates:", oil['date'].nunique())               # Number of unique dates
print("\nDuplicated dates:", oil['date'].duplicated().sum())  # Number of duplicated dates


print("\nHolidays Data:")
print(holidays.head())
print("\nHolidays Data Statistcis:")
print(holidays.describe())
print("\nHolidays Data null values:")
print(holidays.isnull().sum())
print("\nStart date:", holidays['date'].min())  # Earliest date
print("\nEnd date:", holidays['date'].max())    # Latest date
print("\nUnique dates:", holidays['date'].nunique())              # Number of unique dates
print("\nDuplicated dates:", holidays['date'].duplicated().sum()) # Number of duplicated dates - 38 duplicated dates
# View duplicates in holidays
duplicates = holidays[holidays.duplicated(subset='date', keep=False)]
print("Duplicate rows in holidays:")
print(duplicates)
# Fix: Keep the first holiday per date
holidays = holidays.drop_duplicates(subset='date', keep='first')
print("\nStart date:", holidays['date'].min())  # Earliest date
print("\nEnd date:", holidays['date'].max())    # Latest date
print("\nUnique dates:", holidays['date'].nunique())              # Number of unique dates
print("\nDuplicated dates:", holidays['date'].duplicated().sum()) # Number of duplicated dates - no duplicated dates

# As the total number of transactions at the store_nbr and date level with no reliable information to further break it down by family (product class), this table will not be adopted in the training and forecasting process. 
print(transactions.head())
print("\nNumber of duplicate rows:", transactions.duplicated().sum())  # Number of duplicated rows
print("\nStart date:", transactions['date'].min())         # Earliest date
print("\nEnd date:", transactions['date'].max())           # Latest date
print("\nUnique dates:", transactions['date'].nunique())   # Number of unique dates
print("\nDuplicated dates:", transactions['date'].duplicated().sum()) # Number of duplicated dates



Train Data:
   id       date  store_nbr      family  sales  onpromotion
0   0 2013-01-01          1  AUTOMOTIVE    0.0            0
1   1 2013-01-01          1   BABY CARE    0.0            0
2   2 2013-01-01          1      BEAUTY    0.0            0
3   3 2013-01-01          1   BEVERAGES    0.0            0
4   4 2013-01-01          1       BOOKS    0.0            0

Train Data Statistcis:
                 id                           date     store_nbr  \
count  3.000888e+06                        3000888  3.000888e+06   
mean   1.500444e+06  2015-04-24 08:27:04.703088384  2.750000e+01   
min    0.000000e+00            2013-01-01 00:00:00  1.000000e+00   
25%    7.502218e+05            2014-02-26 18:00:00  1.400000e+01   
50%    1.500444e+06            2015-04-24 12:00:00  2.750000e+01   
75%    2.250665e+06            2016-06-19 06:00:00  4.100000e+01   
max    3.000887e+06            2017-08-15 00:00:00  5.400000e+01   
std    8.662819e+05                            NaN  1.558579

Step 2: Merge and Prepare the Data

Merge Datasets

In [42]:
# Merge train with stores data
train = train.merge(stores, on='store_nbr', how='left')

# Merge train with oil prices
train = train.merge(oil, on='date', how='left')

# Merge train with holidays data
train = train.merge(holidays, on='date', how='left')


Check for Data Integrity

In [43]:
# Preview the merged data
print("Merged Train Data:")
print(train.head())
print("\nNumber of rows:", train.shape[0])


# Check for missing values
missing_values = train.isnull().sum()
print("Missing Values:")
print(missing_values)

# Forward fill missing values
train['dcoilwtico'] = train['dcoilwtico'].ffill()
# Backward fill any remaining missing values
train['dcoilwtico'] = train['dcoilwtico'].bfill()
missing_values_1 = train.isnull().sum()

# Verify no missing values for oil prices
print("Missing Values:")
print(missing_values_1) 

# Check for duplicate rows
duplicates = train.duplicated().sum()
print("Number of duplicate rows:", duplicates)

# Preview the merged data
print("Merged Train Data:")
print(train.head())
print("\nNumber of rows:", train.shape[0])

Merged Train Data:
   id       date  store_nbr      family  sales  onpromotion   city      state  \
0   0 2013-01-01          1  AUTOMOTIVE    0.0            0  Quito  Pichincha   
1   1 2013-01-01          1   BABY CARE    0.0            0  Quito  Pichincha   
2   2 2013-01-01          1      BEAUTY    0.0            0  Quito  Pichincha   
3   3 2013-01-01          1   BEVERAGES    0.0            0  Quito  Pichincha   
4   4 2013-01-01          1       BOOKS    0.0            0  Quito  Pichincha   

  type_x  cluster  dcoilwtico   type_y    locale locale_name  \
0      D       13         NaN  Holiday  National     Ecuador   
1      D       13         NaN  Holiday  National     Ecuador   
2      D       13         NaN  Holiday  National     Ecuador   
3      D       13         NaN  Holiday  National     Ecuador   
4      D       13         NaN  Holiday  National     Ecuador   

          description transferred  
0  Primer dia del ano       False  
1  Primer dia del ano       False  
2